In [1]:
from utils import *

In [2]:
# bubble_locations = np.array([[8.0, 3.0, -14.0],[-8.0, 3.0, -14.0]])
bubble_locations = np.array([[8.0, 3.0, -14.0],[8.0, 0.0, -14.0]])
# bubble_locations = np.array([[8.0, 3.0, -14.0]])

sensor1 = create_sensor(sensor_number=1)
sensor2 = create_sensor(sensor_number=2)
sensor3 = create_sensor(sensor_number=3)

pose1 = np.array(sensor1['to_world'].matrix)[:3,:]
pose2 = np.array(sensor2['to_world'].matrix)[:3,:]
pose3 = np.array(sensor3['to_world'].matrix)[:3,:]

poses = np.stack([pose1, pose2, pose3])
inv_poses = compute_inverse_pose_matrices(poses)

# Determine the reprojected pixel positions of the bubbles from each image
normalized_bubble_reprojections = []
for inv_pose in inv_poses:
    reprojs = []
    for loc in bubble_locations:
        reprojs.append(reproject(inv_pose, loc))
    normalized_bubble_reprojections.append(reprojs)

# Convert those re-projections to pixel coordinates
# Note that we do not actually end up using this since `find_correspinding_locations`
# takes in the normalized positions...
bubble_reprojections = []
for projs in normalized_bubble_reprojections:
    reprojs = []
    for proj in projs:
        reprojs.append(normalized_to_pixel(proj))
    bubble_reprojections.append(reprojs)

print(np.array(normalized_bubble_reprojections))

[[[ 1.66697997e-01  1.11875810e-01]
  [ 1.15914514e-01  1.72239412e-01]]

 [[-2.76736935e-01  9.69754544e-02]
  [-2.76737528e-01 -2.13463399e-05]]

 [[ 6.21835210e-02 -2.36068523e-01]
  [ 1.15887333e-01 -1.72234408e-01]]]


In [3]:
def find_corresponding_locations(pose1, pose2, locs1, locs2):
    """Find the corresponding bubbles from two camera views using the pose matrices
    of the cameras and the *normalized* pixel coordinates of the bubbles in the images.
    Corresponding points are found by searching for the minimum ray separations using
    a modified mid-point two view triangulation method.

    Parameters:
        pose1: (3, 4) pose matrix of camera 1\\
        pose2: (3, 4) pose matrix of camera 2\\
        locs1: (n, 2) normalized UV coordinates of bubbles in camera 1\\
        locs2: (n, 2) normalized UV coordinates of bubbles in camera 2

    Returns:
        (n//2, 2, 2) corresponding normalized pixel coordinates of bubbles in cameras 1 and 2
        such that [i,0] is the coordinate of the ith bubble in camera 1 and [i,1] is the 
        corresponding coordinate of the ith bubble in camera 2
    """
    # Extract the rotation matrix and translation vector from the pose matrices
    rmat1 = pose1[:,:3]
    rmat2 = pose2[:,:3]

    # Determine the out-going ray directions
    # Note: We are using -1 for the homogeneous coordinate 
    # bc Mitsuba uses left-handed coordinates!
    ray1ds = []
    for loc in locs1:
        ray1ds.append(rmat1 @ np.array([loc[0], loc[1], -1.0]).T)
    ray2ds = []
    for loc in locs2:
        ray2ds.append(rmat2 @ np.array([loc[0], loc[1], -1.0]).T)

    # The origin of the rays is the translation vectors of the pose matrices
    ray1o = pose1[:,3]
    ray2o = pose2[:,3]

    # Solve for the distance along each pair of rays where they are closest to each other
    closest_distances = []
    for d1 in ray1ds:
        for d2 in ray2ds:
            A = np.array([[d1.T @ d1, -d2.T @ d1],
                          [d1.T @ d2, -d2.T @ d2]])
            b = np.array([(ray2o - ray1o).T @ d1, (ray2o - ray1o).T @ d2])
            ts = np.linalg.inv(A) @ b
            r1t = ray1o + (ts[0] * d1)
            r2t = ray2o + (ts[1] * d2)
            closest_distances.append(np.linalg.norm(r1t - r2t))
    

    # print(closest_distances)
    correspondences = []
    for _ in range(len(closest_distances)//2):
        closest = np.argmin(closest_distances)
        l1 = locs1[closest//2]
        l2 = locs2[closest%2]
        correspondences.append([l1, l2])
        closest_distances[closest] = np.inf
    return np.array(correspondences)

In [4]:
print(find_corresponding_locations(pose1, pose2, normalized_bubble_reprojections[0], normalized_bubble_reprojections[1]))

[[[ 1.15914514e-01  1.72239412e-01]
  [-2.76737528e-01 -2.13463399e-05]]

 [[ 1.66697997e-01  1.11875810e-01]
  [-2.76736935e-01  9.69754544e-02]]]
